<a href="https://colab.research.google.com/github/abimmost/SEED-generative-Ai-with-interview-questions/blob/full-contribute/exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi transformers typing pydantic uvicorn pyngrok

## **ALL** **EXERCISES**

In [ ]:
%%writefile app.py

from fastapi import FastAPI, UploadFile, File, HTTPException
from transformers import pipeline
from typing import Optional
from pydantic import BaseModel

# @title
app = FastAPI()

## Hello LLM Endpoint

class Hello(BaseModel):
    text: str

@app.post("/hello-llm")
def hello(request:Hello):
    hello_pipeline = pipeline("text-generation", model="distilgpt2")
    greeting = hello_pipeline(request.text)
    return {
        "text": request.text,
        "Greetings": greeting
    }

## TEXT SUMMARIZER

class summary(BaseModel):
    long_text: str

@app.post("/summarize")
def summary(request:summary):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summarized = summarizer(
        request.long_text,
        max_length=50,
        min_length=15,
        do_sample=False
    )
    return {
        "Long Text": request.long_text,
        "Summary": summarized
    }

## SENTIMENT ANALYSIS

class Sentiment(BaseModel):
    text: str

@app.post("/sentiment")
def senti(request:Sentiment):
    sentiment_pipeline = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
    senti = sentiment_pipeline(request.text)

    return {
        "text": request.text,
        "sentiment": senti
    }

## MULTIMODAL IMAGE CAPTIONING

@app.post("/caption-image")
def caption_image(file:UploadFile=File(...)):
    caption_pipeline = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
    captioned_image = caption_pipeline(file)

    return {
        "file": file.filename,
        "caption": captioned_image
    }

**SERVING FASTAPI**

In [ ]:
# @title
from pyngrok import ngrok
import uvicorn
import nest_asyncio

# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
auth_token = "33EuUmVjTDuTEVQeCW6lBNFcGQZ_4ZQGynoNCAXRdgFkKSaNi"

# Set the authtoken
ngrok.set_auth_token(auth_token)

# Apply nest_asyncio to allow nested event loops (required for Colab)
nest_asyncio.apply()

# 1. Start the ngrok tunnel on the port where Uvicorn will run
NGROK_TUNNEL = ngrok.connect(8000)
print("Public URL:", NGROK_TUNNEL.public_url)

# # 2. Run Uvicorn
# # The loop will block the notebook, but the URL will be active.
# # Use reload=False in a Colab environment.
# uvicorn.run("app:app", host="0.0.0.0", port=8000, log_level="info")